In [11]:
import numpy as np
import cv2
import csv
import pickle
import matplotlib.pyplot as plt

In [12]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    channel_count = img.shape[2]
    match_mask_color = (255,) * channel_count
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def preprocess(img):
    region_of_interest_vertices = [
    (0, 160),
    (0,60),
    (320, 60),
    (320, 160),
    ]
    cropped_image = region_of_interest(
    img,
    np.array([region_of_interest_vertices], np.int32),
    )
    
    kernel= np.array([[0.6, 2, 0.7],[0, 0 ,0],[-0.5, -2.1, -1]])
    
    cropped_image=cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    filtered_img = cv2.filter2D(cropped_image, -1, kernel)
    
    resized= cv2.resize(filtered_img, (140, 140))    
    return resized

In [13]:
import cv2
import numpy as np
import csv
import pickle
import matplotlib.pyplot as plt

features_directory = r'C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\data\\'
labels_file = r'C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\data\driving_log.csv'


# def preprocess(img):
#     resized = cv2.resize((cv2.cvtColor(img, cv2.COLOR_RGB2HSV))[:, :, 1], (40, 40))
#     return resized


def data_loading(delta):
    logs = []
    features = []
    labels = []
    with open(labels_file, 'rt') as f:
        reader = csv.reader(f)
        for line in reader:
            logs.append(line)
        log_labels = logs.pop(0)

    for i in range(len(logs)):
        for j in range(3):
            img_path = logs[i][j]
            img_path = features_directory + 'IMG' + (img_path.split('IMG')[1]).strip()
            img = plt.imread(img_path)
            features.append(preprocess(img))
            if j == 0:
                labels.append([float(logs[i][j]) for j in range(3,6)])
            elif j == 1:
                labels.append([(float(logs[i][j]) + delta) for j in range(3,6)])
            else:
                labels.append([(float(logs[i][j]) - delta) for j in range(3,6)])
    return features, labels


delta = 0.2
features, labels = data_loading(delta)

features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')

with open(r"C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\data\\features_40", "wb") as f:
    pickle.dump(features, f, protocol=4)
with open(r"C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\data\\labels", "wb") as f:
    pickle.dump(labels, f, protocol=4)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, Dropout
from keras.utils import print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pickle
from keras.optimizers import Adam


def keras_model():
    model = Sequential()
    model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(140, 140, 1)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))

    model.add(Flatten())
    model.add(Dropout(0.5))

    model.add(Dense(128))

    model.add(Dense(64))
    model.add(Dense(3))

    model.compile(optimizer=Adam(lr=0.0001), loss="mse")
    filepath = r"C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\Autopilot.h5_v1"
    checkpoint1 = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
    callbacks_list = [checkpoint1]

    return model, callbacks_list

# model, callbacks_list = keras_model()
# print_summary(model)

def loadFromPickle():
    with open(r"C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\data\\features_40", "rb") as f:
        features = np.array(pickle.load(f))
    with open(r"C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\data\\labels", "rb") as f:
        labels = np.array(pickle.load(f))

    return features, labels


def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels


def main():
    features, labels = loadFromPickle()
    features, labels = augmentData(features, labels)
    features, labels = shuffle(features, labels)
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,
                                                        test_size=0.1)
    print(train_y[0])
    plt.imshow(train_x[0])
    plt.show()
    train_x = train_x.reshape(train_x.shape[0], 140, 140, 1)
    test_x = test_x.reshape(test_x.shape[0], 140, 140, 1)
    model, callbacks_list = keras_model()
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=5, batch_size=32,
              callbacks=callbacks_list)
    print_summary(model)
    model.save('Autopilot.h5')


main()


In [10]:
from keras.models import load_model

model = load_model(r"C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\Autopilot.h5_v1")
img = cv2.imread(r"C:\Users\nishc_omjn2ty\Downloads\trial5.jpg")
processed = preprocess(img)
cv2.imshow("img", processed)
cv2.waitKey(0)
# p = np.array(processed, dtype=np.float32)
# p = np.reshape(p, (-1, 140, 140, 1))
# print(model.predict(p, batch_size=1))

-1

In [9]:
# import numpy as np
# import cv2
from keras.models import load_model

model = load_model('Autopilot.h5')

def keras_predict(model, image):
    processed = preprocess(image)
    #p = np.array(processed, dtype=np.float32)
    p = np.reshape(processed, (-1,140, 140, 1))
    steering_angle = float(model.predict(p, batch_size=1)[0])
    steering_angle = steering_angle * 100
    return steering_angle

steer = cv2.imread(r'C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\\steering_wheel_image.jpg',0)
rows, cols = steer.shape
smoothed_angle = 0

cap = cv2.VideoCapture(r'C:\Users\nishc_omjn2ty\Downloads\End-to-End\End-to-End-Learning-for-Self-Driving-Cars-master\Implementation_1\\run.mp4')
while (cap.isOpened()):
    ret, frame = cap.read()
    #gray = cv2.resize((cv2.cvtColor(frame, cv2.COLOR_RGB2HSV))[:, :, 1], (40, 40))
    steering_angle = keras_predict(model, frame)
    print(steering_angle)
    cv2.imshow('frame', cv2.resize(frame, (500, 300), interpolation=cv2.INTER_AREA))
    smoothed_angle += 0.2 * pow(abs((steering_angle - smoothed_angle)), 2.0 / 3.0) * (
        steering_angle - smoothed_angle) / abs(
        steering_angle - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), -smoothed_angle, 1)
    dst = cv2.warpAffine(steer, M, (cols, rows))
    cv2.imshow("steering wheel", dst)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


-6.847521662712097
-6.832114607095718
-6.83353915810585
-6.786021590232849
-6.970445066690445
-6.970445066690445
-7.24458321928978
-7.283744215965271
-7.395411282777786
-7.446588575839996
-7.503547519445419
-7.491208612918854
-6.643831729888916
-6.694890558719635
-6.701196730136871
-7.266918569803238
-7.266745716333389
-7.331849634647369
-8.60508531332016
-8.612830936908722
-8.620937168598175
-9.057427942752838
-8.986936509609222
-9.035765379667282
-6.839929521083832
-6.790418922901154
-6.751096993684769
-6.441186368465424
-6.44354522228241
-6.458696722984314
-5.224459245800972
-5.2275244146585464
-5.223392695188522
-5.257765203714371
-5.257765203714371
-5.279122292995453
-5.36092147231102
-3.0433617532253265
-3.03876381367445
-3.0309712514281273
-2.1610310301184654
-3.5664867609739304
-3.546079620718956
-4.733002930879593
-4.733023047447205
-4.749516397714615
-4.970518499612808
-4.996826499700546
-5.026081204414368
-3.9341211318969727
-3.924407809972763
-3.892870619893074
-3.305201977

-2.3856021463871
-2.3385223001241684
-2.234344370663166
-1.3285331428050995
-1.3222726061940193
-1.3142528012394905
-2.1142682060599327
-2.114281617105007
-2.137509174644947
-1.6822680830955505
-1.6793863847851753
-1.620662584900856
-1.171521283686161
-0.13346605701372027
-0.3374094609171152
-0.9195364080369473
-0.9195364080369473
-0.920209102332592
-0.2299262909218669
-0.21714128088206053
-0.2351144328713417
0.8674084208905697
0.8759908378124237
0.8662153035402298
0.41857706382870674
-1.2318173423409462
-1.100410521030426
1.13444859161973
1.1499292217195034
1.1183691211044788
0.33743500243872404
0.3034325782209635
0.2924948697909713
-0.8798031136393547
-0.8798031136393547
-0.8783575147390366
-0.5763084162026644
-0.5936367437243462
-0.5927002988755703
0.029871618608012795
-0.001111922028940171
0.06120277103036642
-0.15322769759222865
-0.1508931629359722
-0.1512146322056651
-0.7827586494386196
-0.8185883983969688
-0.7827586494386196
-0.7824696600437164
-0.7572410628199577
-0.82937330007

1.4186584390699863
1.4536235481500626
1.4080384746193886
2.2347619757056236
2.320760488510132
2.346329391002655
-1.3574435375630856
-1.3603010214865208
-1.4099622145295143
-3.542981296777725
-3.5835590213537216
-3.535938635468483
-2.6730114594101906
-2.7910031378269196
-2.9082216322422028
-2.9273055493831635
-2.9147733002901077
-2.9003366827964783
-2.2854216396808624
-2.2742902860045433
-2.3483457043766975
-1.3115568086504936
-1.266875583678484
-1.2135833501815796
-1.812916062772274
-1.7121551558375359
-1.8492339178919792
-0.7867877371609211
-0.76998146250844
-0.7635662332177162
-0.5876225419342518
-0.5876225419342518
-0.7364459335803986
0.419611856341362
0.28972746804356575
0.3368645906448364
0.3745339345186949
0.42281029745936394
0.39659496396780014
1.0056287981569767
0.9599313139915466
0.9563121944665909
0.9455162100493908
0.9614436887204647
0.8540733717381954
1.5739189460873604
1.5580730512738228
1.3973879627883434
-1.5179337933659554
-1.5289449132978916
-1.5396922826766968
0.60381

-2.7605732902884483
-2.7717657387256622
-2.714037150144577
-6.196437403559685
-6.228633597493172
-6.276626139879227
-2.9432283714413643
-2.986159175634384
-3.0233345925807953
-3.515228256583214
-3.490878641605377
-3.4954283386468887
-3.448309004306793
-3.4448709338903427
-3.4117959439754486
-3.253702074289322
-3.15946564078331
-3.162473812699318
-2.8030654415488243
-2.8237801045179367
-2.839179150760174
-3.358844295144081
-3.35521399974823
-3.348778188228607
-4.7525059431791306
-4.747260361909866
-4.702386632561684
-5.4862406104803085
-5.4862406104803085
-5.4862406104803085
-5.004733055830002
-5.011952295899391
-5.004733055830002
-6.978724151849747
-6.988825649023056
-7.001076638698578
-5.453412979841232
-5.389506742358208
-5.383602902293205
-4.628429934382439
-4.613514989614487
-4.583831876516342
-5.680633336305618
-5.623102933168411
-5.533711984753609
-4.175331816077232
-4.157045483589172
-4.135213792324066
-5.38359098136425
-5.3459495306015015
-5.38908876478672
-5.521197617053986
-5

-3.1552087515592575
-3.2343000173568726
-4.075460508465767
-4.101790115237236
-4.0572389960289
-6.13694041967392
-6.17915466427803
-6.2067970633506775
-4.99502457678318
-4.99502457678318
-4.99502457678318
-6.1873659491539
-6.203748285770416
-6.1809733510017395
-6.966116279363632
-6.9774821400642395
-6.9744594395160675
-7.017287611961365
-7.052019238471985
-7.095912843942642
-7.416680455207825
-7.412577420473099
-7.412310689687729
-6.348766386508942
-6.336885690689087
-6.280577182769775
-6.48968368768692
-7.406230270862579
-7.469136267900467
-6.100068986415863
-6.084369868040085
-6.060977652668953
-6.130418553948402
-6.140660122036934
-6.16784542798996
-4.6883996576070786
-4.70963642001152
-4.667748883366585
-6.684040278196335
-6.6864050924777985
-6.72849714756012
-6.592739373445511
-6.669316440820694
-6.669316440820694
-7.365227490663528
-7.300320267677307
-7.343224436044693
-5.091036111116409
-5.084048211574554
-5.084048211574554
-4.694471135735512
-4.707501456141472
-4.71776165068149

...

0.08648597868159413
0.1727344118990004
0.17183251911774278
1.7077285796403885
1.7094491049647331
1.7077285796403885
1.136971078813076
1.1413725093007088
1.1268764734268188
1.7008192837238312
1.2830432504415512
1.2784468941390514
3.1154338270425797
3.1381893903017044
3.1435199081897736
-1.013981644064188
-1.013981644064188
-1.0400197468698025
-2.496938407421112
-2.52737570554018
-2.581448294222355
-2.088494226336479
-2.703234553337097
-2.76626106351614
-2.2152256220579147
-2.3011213168501854
-2.1104156970977783
-3.284306824207306
-3.2081451267004013
-3.284306824207306
-2.447909861803055
-2.4715280160307884
-2.4594349786639214
-3.3997032791376114
-3.3708736300468445
-3.3735238015651703
-1.8513115122914314
-1.3678421266376972
-1.3130618259310722
-1.8916923552751541
-1.8916923552751541
-1.8916923552751541
-2.4159640073776245
-2.415977604687214
-2.424345165491104
-0.8424844592809677
-0.8558144792914391
-0.8424844592809677
-0.5248900502920151
-0.5228674039244652
-0.5481421947479248
-1.308582

-5.972254276275635
-5.995883792638779
-5.868393927812576
-5.735631287097931
-5.73611855506897
-5.746375024318695
-4.1349489241838455
-4.128369316458702
-4.295680299401283
-5.278065428137779
-5.294311046600342
-5.259747430682182
-5.394152924418449
-5.396386981010437
-5.394152924418449
-6.492076814174652
-6.489597260951996
-6.495861709117889
-7.8695885837078094
-7.864631712436676
-7.8313663601875305
-7.201187312602997
-7.2033606469631195
-7.215914875268936
-8.728635311126709
-8.826922625303268
-8.856236189603806
-8.28862413764
-8.334240317344666
-8.330809324979782
-6.656476855278015
-6.656476855278015
-6.669595092535019
-3.295671194791794
-3.321082517504692
-3.35790291428566
-3.3866703510284424
-3.377373144030571
-3.4001443535089493
-6.153113394975662
-6.142986938357353
-6.1485327780246735
-6.874746084213257
-6.874746084213257
-6.8047091364860535
-4.703083634376526
-4.6540141105651855
-4.632977768778801
-5.736887082457542
-5.716868117451668
-5.788500979542732
-5.743308365345001
-5.709922

-6.198522821068764
-6.301781535148621
-4.195893183350563
-4.18124832212925
-4.15639728307724
-4.64390441775322
-4.660293459892273
-4.629950970411301
-4.4869717210531235
-4.481194540858269
-4.489150643348694
-4.763542488217354
-4.811931028962135
-4.760513454675674
-5.472120642662048
-5.467146262526512
-5.42660765349865
-6.270814687013626
-6.291921436786652
-6.281694024801254
-4.125265404582024
-4.087799787521362
-4.214248806238174
-4.99008372426033
-5.007617548108101
-4.986801743507385
-5.839981883764267
-5.825820937752724
-5.825141444802284
-5.975062400102615
-6.003273651003838
-6.016545370221138
-8.120806515216827
-8.124513179063797
-8.12576487660408
-7.54118412733078
-7.419773191213608
-7.483293116092682
-4.272815957665443
-4.134209826588631
-4.0400538593530655
-4.426397383213043
-4.426397383213043
-4.4539764523506165
-5.855084583163261
-5.800968408584595
-5.825796723365784
-4.471715912222862
-4.530884325504303
-4.354677721858025
-6.734146177768707
-6.713548302650452
-6.6938623785972

-1.0591408237814903
-1.094738394021988
-1.0985198430716991
-0.23669584188610315
-0.23669584188610315
-0.22412422113120556
-1.1164650321006775
-1.11435791477561
-1.1146518401801586
-2.898026630282402
-2.931709587574005
-2.898026630282402
-1.4494160190224648
-1.36783542111516
-1.4057418331503868
-1.076911948621273
-0.855711754411459
-0.855711754411459
-0.19958727061748505
-0.19108819542452693
-0.18070702208206058
0.692498404532671
0.14706079382449389
0.14706079382449389
0.43257097713649273
0.42937970720231533
0.3639198839664459
0.4070138093084097
0.37033832632005215
0.30488260090351105
-0.05135142710059881
-2.329070121049881
-2.1267807111144066
0.9631417691707611
0.9780301712453365
0.9596530348062515
0.3801889717578888
0.39515094831585884
0.44110282324254513
1.1982138268649578
1.1888707987964153
1.1864056810736656
-0.058697303757071495
-0.058697303757071495
-0.058697303757071495
0.1398562453687191
0.13147128047421575
0.08495530346408486
-0.06224209209904075
-0.07591246394440532
0.0708586

AttributeError: 'NoneType' object has no attribute 'shape'